## Ingest circuits.csv file

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configurations"

In [0]:
%run "../includes/common_functions"

**Step 1: Read the CSV file using spark dataframe reader**

In [0]:
display(dbutils.fs.ls(f"{raw_folder_path}"))

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType 

In [0]:
# StructType is representing rows
# StructFields is representing columns
circuit_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                    StructField("circuitRef", StringType(), True),
                                    StructField("name", StringType(), True),
                                    StructField("location", StringType(), True),
                                    StructField("country", StringType(), True),
                                    StructField("lat", DoubleType(), True),
                                    StructField("lng", DoubleType(), True),
                                    StructField("alt", IntegerType(), True),
                                    StructField("url", StringType(), True)
                                    ])

In [0]:
circuits_df = spark.read \
    .option("header", True) \
        .schema(circuit_schema) \
            .csv(f"{raw_folder_path}/circuits.csv")

In [0]:
circuits_df.limit(5).display()

In [0]:
display(circuits_df.printSchema())
# the standard behaviour of the dataframe reader, if we need to handle nulls,
# that needs to be handled explicitly using other functions

**Step 2: Selecting only the required columns**

In [0]:
from pyspark.sql.functions import col, lit

circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"),
                                          col("location"), col("country"), col("lat"), col("lng"), col("alt"))

circuits_selected_df.limit(5).display()

**Step 3: Renaming the columns**

In [0]:
circuits_renanmed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
                                            .withColumnRenamed("circuitRef", "circuit_ref") \
                                            .withColumnRenamed("lat", "latitude") \
                                            .withColumnRenamed("lng", "longitude") \
                                            .withColumnRenamed("alt", "altitude") \
                                            .withColumn("data_source", lit(v_data_source))    



In [0]:
circuits_renanmed_df.limit(5).display()

**Step 4: adding a new column ingestion timestamp**

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
circuits_final_df = ingestion_date(circuits_renanmed_df)

In [0]:
circuits_final_df.limit(5).display()

**Step 5: Now write the final csv file to the processed container**

In [0]:
circuits_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits")

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/circuits"))

In [0]:
dbutils.notebook.exit("Success")